In [1]:
import os
import pandas as pd
import json
import pickle
import zipfile
import shutil

In [ ]:
# 📌 Desinstalar posibles versiones conflictivas
!pip uninstall -y tensorflow keras protobuf

# 📌 Instalar versiones compatibles
!pip install tensorflow==2.12.0 keras==2.12.0 protobuf==3.20.3


In [2]:

# 📌 Instalar dependencias de la API
!pip install tf_slim pycocotools contextlib2 pillow lxml matplotlib pandas opencv-python-headless Cython tensorflow_io apache-beam

# 📌 Instalar compilador protobuf
!apt-get install -y protobuf-compiler

# 📌 Clonar repositorio oficial
!git clone https://github.com/tensorflow/models.git

# 📌 Compilar archivos .proto
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

# 📌 Instalar la API sin actualizar dependencias
!cp object_detection/packages/tf2/setup.py .
!python -m pip install . --no-deps

# 📌 Exportar PYTHONPATH
import os
os.environ['PYTHONPATH'] += ':/content/models:/content/models/research:/content/models/research/slim'

# 📌 Verificar instalación
!python object_detection/builders/model_builder_tf2_test.py


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697324 sha256=5d57ef91e4ac610b69e433c4b0cbde6633b177f2448a84ea4788c127409d983d
  Stored in directory: /tmp/pip-ephem-wheel-cache-t9cddo1i/wheels/95/4a/63/b2d36ca06eab841de19a38993ffaf2beac152a44539bc642a6
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1
2025-05-24 02:10:44.019417: I tensorflow/tsl/cuda/cudart_stub.cc:

In [5]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: apache-beam, google-ai-generativelanguage, google-api-core, google-cloud-aiplatform, google-cloud-bigquery-connection, google-cloud-bigquery-storage, google-cloud-dataproc, google-cloud-datastore, google-cloud-firestore, google-cloud-functions, google-cloud-iam, google-cloud-language, google-cloud-resource-manager, google-cloud-spanner, google-cloud-translate, google-generativeai, googleapis-common-protos, grpc-google-iam-v1, grpcio-status, kaggle, orbax-checkpoint, proto-plus, tb-nightly, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata, tf-hub-nightly, tf_nightly, wandb, ydf, yfinance


In [ ]:
labels = [{'name':'carro', 'id': 1}, {'name': 'motos', 'id': 2}]
with open("label_map.pbtxt", "w") as f:
  for label in labels:
    f.write('item { \n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

In [ ]:
#Descargar los modelos pre-entrenados en este caso SSD + MobileNetV2
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
-O /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2025-05-23 23:46:55--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.24.207, 142.251.10.207, 142.251.12.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.24.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

/content/ssd_mobile 100%[===================>]  19.56M  9.46MB/s    in 2.1s    

2025-05-23 23:46:57 (9.46 MB/s) - ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [6]:
# descomprimir el archivo .gz
#!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
output_path = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almaceno en {}".format(output_path))

La carpeta se almaceno en /content/models/research/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


In [9]:
# crear carpeta de entrenamineto
path_training = '/content/ssd_mobilenet'
#os.mkdir(path_training)

In [ ]:
# mover el archivo pipeline.config de la carpeta ssd_mobilenet_v2.... hacia la carpeta ssd_mobilenet
shutil.move('/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', path_training)

Error: Destination path '/content/ssd_mobilenet/pipeline.config' already exists

In [7]:
import tensorflow as tf
from google.protobuf import text_format

In [5]:
from object_detection.protos import pipeline_pb2


In [10]:
# crear el archivo de configuracio para que lo podamos editar

pipeline = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(os.path.join(path_training, 'pipeline.config'), 'r') as f:
  proto_str = f.read()
  # Remove the problematic field before parsing
  proto_str = proto_str.replace('fine_tune_checkpoint_version: V2', '')
  text_format.Merge(proto_str, pipeline)

In [11]:

label_map_path = '/content/label_map.pbtxt'
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'

In [12]:
# Editar el archivo config de la arquitectura de mobilenet_v2

# Establecer el numero de clases
pipeline.model.ssd.num_classes = 2

# Establecer la cantidad del lote
pipeline.train_config.batch_size = 4

# Definimos desde donde quiero que empieze a entrenar , desde que checkpoint
pipeline.train_config.fine_tune_checkpoint = os.path.join(output_path, 'checkpoint/ckpt-0')

# Definir que tipo de entrenamiento queremos hacer
pipeline.train_config.fine_tune_checkpoint_type = 'detection'


#Obtener el path del label_map de train
pipeline.train_input_reader.label_map_path = label_map_path

#Leer el archivo tf record train
pipeline.train_input_reader.tf_record_input_reader.input_path[0] = train_record_path

# Obtener el path del label_map de test
pipeline.eval_input_reader[0].label_map_path = label_map_path

#Leer el archivo tf record test
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_path










In [13]:
# sobre escribimos el arvhio pipeline en el archivo pipeline.config dentro de la carpeta ssd_mobilenet
config_text = text_format.MessageToString(pipeline)
with tf.io.gfile.GFile(os.path.join(path_training, 'pipeline.config'), 'wb') as f:
  f.write(config_text)

In [14]:
# Entrenar el modelo
#Definicion de parametros

num_steps = 5000
num_eval_steps = 500
model_dir = '/content/ssd_mobilenet'

# Script de tensorflow  para entrener el modelo con la API DE Open detection

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={path_training}/pipeline.config \
--num_train_steps={num_steps} \
--model_dir={model_dir}

2025-05-24 02:05:58.895871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "/content/models/research/object_detection/model_lib_v2.py", line 30, in <module>
    from object_detection import inputs
  File "/content/models/research/object_detection/inputs.py", line 27, in <module>
    from object_detection.builders import model_builder
  File "/content/models/research/object_detection/builders/model_builder.py", line 70, in <module>
    from object_detection.models import ssd_efficientnet_bifpn_feature_extractor as ssd_efficientnet_bifpn
  File "/content/models/research/object_detection/models/ssd_efficientnet_bifpn_feature_extractor.py", line 35, in <module>
    from official.legacy.image_classification.efficientnet import efficientnet_model
  File "/content/mode

In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf-models-official, tf_keras


In [ ]:
# Downgrade protobuf to a compatible version
!pip uninstall protobuf -y
!pip install protobuf==3.20.x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
# Uninstall current TensorFlow version
!pip uninstall tensorflow -y

# Install TensorFlow version 2.15.0
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0
  Attempting uninstall: tensorboard
    Found existing installat

In [ ]:
!pip uninstall tensorflow-object-detection-api -y

Found existing installation: tensorflow_object_detection_api 0.1.1
Uninstalling tensorflow_object_detection_api-0.1.1:
  Successfully uninstalled tensorflow_object_detection_api-0.1.1


In [ ]:
!pip install tensorflow_io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.7 MB/s eta 0:00:00


In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research
